In [3]:
# import matplotlib.pyplot as plt
# import numpy as np
# %matplotlib widget

# # Fixing random state for reproducibility
# np.random.seed(19680801)

# fig, axs = plt.subplots(2, 2)

# axs[0, 0].imshow(np.random.random((100, 100)))

# axs[0, 1].imshow(np.random.random((100, 100)))

# axs[1, 0].imshow(np.random.random((100, 100)))

# axs[1, 1].imshow(np.random.random((100, 100)))

# plt.subplot_tool()

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics.cluster import *
from sklearn.metrics import *
import scanpy as sc
from scipy.optimize import linear_sum_assignment
import numpy as np
import umap
import phate
%matplotlib widget

In [2]:
# for hg in hgs:
#     for dim in dims:
#         monocle2_cluster_file = "hg%s.monocle2.cluster.txt" % hg
#         monocle2_anno_file = "hg%s.monocle2.anno.txt" % hg
#         for dataset_dir in dataset_dirs:
#             ref_anno = np.loadtxt(os.path.join(data_dir, dataset_dir, "0.1.Y.csv"), dtype=int)
#             cluster = np.genfromtxt(os.path.join(data_dir, dataset_dir, output_dir, monocle2_cluster_file), dtype=int)
#             anno = np.loadtxt(os.path.join(data_dir, dataset_dir, output_dir, monocle2_anno_file), dtype=int)
#             assert anno[anno.shape[0] // 2:].shape == cluster[cluster.shape[0] // 2:].shape
#             assert anno[anno.shape[0] // 2:].shape[0] == ref_anno.shape[0] * 10
#             np.savetxt(os.path.join(data_dir, dataset_dir, output_dir, monocle2_cluster_file), cluster[cluster.shape[0] // 2:], fmt="%d")
#             np.savetxt(os.path.join(data_dir, dataset_dir, output_dir, monocle2_anno_file), cluster[anno.shape[0] // 2:], fmt="%d")

In [2]:
def recover_labels(y_true, y_pred):
    unique_cluster, y_pred = np.unique(y_pred, return_inverse=True)
#     print(np.unique(unique_cluster))
    # if a method cannot produce results, assign the worst result for it
    if len(unique_cluster) == 1:
        print("This method cannot produce cluster at all.")
        for i in range(len(np.unique(y_true))):
            y_pred[i] = i
#     print("idx: ", np.unique(y_pred))
    counting_mat = contingency_matrix(y_true, y_pred)
#     # print(counting_mat)
#     y_pred_anno_idx = counting_mat.argmax(axis=0)
#     # annotate back can only supporting corresponding clustering number
#     y_pred_anno_dict = {i: y_pred_anno_idx[i] for i in range(len(y_pred_anno_idx))}
#     #y_pred_anno_dict = {i: y_pred_anno_idx[i] for i in np.unique(y_pred)}
#     y_pred_anno = np.array([y_pred_anno_dict[k] for k in y_pred])
    row_idx, col_idx = linear_sum_assignment(counting_mat.max() - counting_mat)
#     print(row_idx, col_idx)
    y_pred_anno_dict = {c: r for r, c in zip(row_idx, col_idx)}
#     from munkres import Munkres
#     m = Munkres()
#     indices = m.compute(counting_mat.max() - counting_mat)
#     print("indices: ",indices)
#     Y_pred_anno_dict = {column: row for row,column in indices}
    y_pred_anno = np.array([y_pred_anno_dict[k] for k in y_pred])
    return y_pred_anno

In [3]:
#Evaluation Function
def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    counting_mat = contingency_matrix(y_true, y_pred)
    return np.sum(np.amax(counting_mat, axis=0)) / np.sum(counting_mat)

In [4]:
def average_silhouette_width(X, Y, batch):
    asw_batch = silhouette_score(X, batch)
    asw_celltype = silhouette_score(X, Y)
    min_val = -1
    max_val = 1
    asw_batch_norm = (asw_batch - min_val) / (max_val - min_val)
    asw_celltype_norm = (asw_celltype - min_val) / (max_val - min_val)
    return (2 * (1 - asw_batch_norm)*(asw_celltype_norm))/(1 - asw_batch_norm + asw_celltype_norm)

In [5]:
#dataset_dirs = ["human_pancreas", "human_melanoma", "mouse_stem", "mouse_pancreas", "human_progenitor", "mouse_cortex"]
dataset_dirs = ["human_progenitor", "human_colorectal_cancer", "mouse_cortex", "mouse_stem", "human_melanoma", "human_pancreas", "mouse_pancreas", "mouse_blastomeres", "mouse_pancreatic_circulating_tumor", "human_embryos", "splatter", 'mars_skin', "mars_limb_muscle", 'mars_tongue', 'mars_bladder', 'mars_liver', 'mars_trachea', 'mars_mammary_gland', 'mars_spleen'] #]
dataset_dirs = ["human_pancreas", "mouse_pancreas", "human_melanoma", "mouse_stem", "mouse_cortex", "human_progenitor", 'mars_skin', "mars_limb_muscle", 'mars_bladder', 'mars_trachea', 'mars_mammary_gland', 'mars_spleen']
# splatter_dataset_dirs = ["splatter_equal_group_2", "splatter_equal_group_3", "splatter_equal_group_4", "splatter_equal_group_5", "splatter_equal_group_6", "splatter_equal_group_7", "splatter_equal_group_8", "splatter_equal_group_9", "splatter_equal_group_10"]
# splatter_dataset_dirs = ["splatter_each_group_2", "splatter_each_group_3", "splatter_each_group_4", "splatter_each_group_5", "splatter_each_group_6", "splatter_each_group_7", "splatter_each_group_8", "splatter_each_group_9", "splatter_each_group_10"]
# splatter_dataset_dirs = ["splatter_each_group_10"]
# splatter_dataset_dirs = ["splatter_dropout_11", "splatter_dropout_10", "splatter_dropout_9", "splatter_dropout_8", "splatter_dropout_7", "splatter_dropout_6", "splatter_dropout_5", "splatter_dropout_4", "splatter_dropout_3", "splatter_dropout_2"]
splatter_dataset_dirs = ["splatter_dropout_3"]
dataset_dirs = splatter_dataset_dirs

dataset_dirs = ["splatter_batch_2", "splatter_batch_3", "splatter_batch_4"] #, "splatter_batch_5"]
# dataset_dirs = ["splatter_batch_5"]
dataset_dirs_batch = ["human_pancreas", "mouse_pancreas", "human_colorectal_cancer"]
dataset_dirs_batch += ["splatter_batch_2", "splatter_batch_3", "splatter_batch_4", "splatter_batch_5"]
methods_non_deterministic_cluster_num = ["seurat", "scvi"]

#dataset_dirs = ['mars_skin', 'mars_tongue', 'mars_bladder', 'mars_liver']
#dataset_dirs = ["mars_limb_muscle", 'mars_trachea', 'mars_mammary_gland']
# extreme good performance datasets: ["mars_limb_muscle", 'mars_mammary_gland', 'mars_bladder']
#--
# good performance datasets: ["mars_trachea", 'mars_skin']
# just-so-so performance datasets: ["mars_liver", 'mars_tongue']
#--
# low performance datasets: ['mars_thymus', "mouse_blastomeres", "human_cell_line", "human_colorectal_cancer", "mouse_embryos", "mouse_pancreatic_circulating_tumor", "human_embryos"]
# 

In [6]:
len(dataset_dirs)

3

In [7]:
data_dir = "F:/OneDrive - Hong Kong Baptist University/year1_1/cgi_datasets"
data_dir = "E:/OneDrive - stu.hit.edu.cn/year1_1/cgi_datasets"
output_dirs = ["output"]
output_dirs = ["test-batch"]
output_dirs = ["test-input_count_X_output_count_X"]
original_output_dir = output_dirs[0]
# output_dirs = ["test-n_init"]

# lambda_bs = ["0", "0.2", "0.4", "0.6", "0.8", "1.0"]
# lambda_cs = ["0", "0.2", "0.4", "0.6", "0.8", "1.0"]

# output_dirs = []
# for lambda_b in lambda_bs:

#     for lambda_c in lambda_cs:
#         output_dirs.append("test-entropy-mask-lambda_b_%s-lambda_c_%s" %(lambda_b, lambda_c))



In [8]:
hgs = [str(0.1)]
hgs = list(map(str, [x / 10.0 for x in range(5, 8)]))
hgs = list(map(str, [x / 10.0 for x in range(1, 3)]))
# dims = [25, 100, 400, 1000]
dims = [400]
dim = 400
repeat_num = 10
repeat_num = 1
plot_metric = 'ARI'
plot_flag = True

In [9]:
for output_dir in output_dirs:
    for hg in hgs:
        for dim in dims:
            # all methods
            group_names = ["%s-%d-inherit" % (hg, dim), "%s-%d-reinit" % (hg, dim), "%s-%d-inherit-L2" % (hg, dim), "%s-%d-reinit-L2" % (hg, dim), "RaceID3", "SC3", "Monocle2", "Seurat3", "SIMLR", "DRjCC", "scvi"]
            # our methods
#             group_names = ["%s-%d-inherit" % (hg, dim), "%s-%d-reinit" % (hg, dim), "%s-%d-inherit-L2" % (hg, dim), "%s-%d-reinit-L2" % (hg, dim)]

            inherit_cluster_file = "hg%s-dim%d-inherit-KMeans.cluster.txt" % (hg, dim)
    #         inherit_anno_file = "hg%s-dim%d-inherit-KMeans.anno.txt" % (hg, dim)
            inherit_l2_cluster_file = "hg%s-dim%d-inherit-KMeans-L2.cluster.txt" % (hg, dim)
    #         inherit_l2_anno_file = "hg%s-dim%d-inherit-KMeans-L2.anno.txt" % (hg, dim)
            reinit_cluster_file = "hg%s-dim%d-reinit-KMeans.cluster.txt" % (hg, dim)
    #         reinit_anno_file = "hg%s-dim%d-reinit-KMeans.anno.txt" % (hg, dim)
            reinit_l2_cluster_file = "hg%s-dim%d-reinit-KMeans-L2.cluster.txt" % (hg, dim)
    #         reinit_l2_anno_file = "hg%s-dim%d-reinit-KMeans-L2.anno.txt" % (hg, dim)
            raceid3_cluster_file = "hg%s.raceid3.cluster.txt" % hg
    #         raceid3_anno_file = "hg%s.raceid3.anno.txt" % hg
            sc3_cluster_file = "hg%s.sc3.cluster.txt" % hg
    #         sc3_anno_file = "hg%s.sc3.anno.txt" % hg
            monocle2_cluster_file = "hg%s.monocle2.cluster.txt" % hg
    #         monocle2_anno_file = "hg%s.monocle2.anno.txt" % hg
            seurat3_cluster_file = "hg%s.seurat.cluster.txt" % hg
    #         seurat3_anno_file = "hg%s.seurat.anno.txt" % hg
            simlr_cluster_file = "hg%s.simlr.cluster.txt" % hg
    #         simlr_anno_file = "hg%s.simlr.anno.txt" % hg
            drjcc_cluster_file = "hg%s.drjcc.cluster.txt" % hg
#             drjcc_anno_file = "hg%s.drjcc.anno.txt" % hg
            scvi_cluster_file = "hg%s.scvi.cluster.txt" % hg
#             scvi_anno_file = "hg%s.scvi.anno.txt" % hg            
            X_file = "%s.X.count.name.csv" % hg
            X_unscaled_file = "%s.X.name.csv" % hg
            anno_file = "%s.Y.csv" % hg
            # all methods
            cluster_files = [inherit_cluster_file, reinit_cluster_file, inherit_l2_cluster_file, reinit_l2_cluster_file, raceid3_cluster_file, sc3_cluster_file, monocle2_cluster_file, seurat3_cluster_file, simlr_cluster_file, drjcc_cluster_file, scvi_cluster_file]
            # our methods
#             cluster_files = [inherit_cluster_file, reinit_cluster_file, inherit_l2_cluster_file, reinit_l2_cluster_file]

            batch_file = "%s.batch.csv" % hg
    #         anno_files = [inherit_anno_file, reinit_anno_file, inherit_l2_anno_file, reinit_l2_anno_file, raceid3_anno_file, sc3_anno_file, monocle2_anno_file, seurat3_anno_file, simlr_anno_file]
    #         anno_files = [anno_file] * len(cluster_files)
    #         X_files = [X_file] * len(cluster_files)
            for dataset_dir in dataset_dirs:
                df = pd.DataFrame({'method': [g for g in group_names for i in range(repeat_num)]})
                anno = np.loadtxt(os.path.join(data_dir, dataset_dir, anno_file), dtype=int)
                X = pd.read_csv(os.path.join(data_dir, dataset_dir, X_file), index_col=0)
                X_unscaled = pd.read_csv(os.path.join(data_dir, dataset_dir, X_unscaled_file), index_col=0)
                adata = sc.AnnData(X_unscaled)
                sc.pp.scale(adata)
                adata = adata.copy()
                X_normalized = adata.X
                if dataset_dir in dataset_dirs_batch:
                    sc_batch_labels = np.loadtxt(os.path.join(data_dir, dataset_dir, batch_file), dtype=int)
                AMI = []
                ARI = []
    #             CPS = []
                FMI = []
                NMI = []
                JACCARD = []
    #             PURITY = []
                ASW = []
                ACC = []
                plot_metric_df = pd.DataFrame()
                plot_metric_df['gt'] = anno
                plot_metric_df['gt_unscaled'] = anno
                plot_metric_df['gt_normalized'] = anno

                for cluster_file, group_name in zip(cluster_files, group_names):
                    output_dir = original_output_dir
                    if group_name.endswith("inherit"):
                        plot_metric_df['gt_imX_unscaled_inherit'] = anno
                    elif group_name.endswith("reinit"):
                        plot_metric_df['gt_imX_unscaled_reinit'] = anno
                    elif group_name.endswith("inherit-L2"):
                        plot_metric_df['gt_imX_unscaled_inherit_L2'] = anno
                    elif group_name.endswith("reinit-L2"):
                        plot_metric_df['gt_imX_unscaled_reinit_L2'] = anno
                    # !!!
                    else:
                        output_dir = "test-batch"
                    print(cluster_file)
                    cluster = np.genfromtxt(os.path.join(data_dir, dataset_dir, output_dir, cluster_file), dtype=int, filling_values=1)
                    for rep in range(repeat_num):
                        one_cluster = cluster[len(cluster)//repeat_num * rep:len(cluster)//repeat_num * (rep + 1)]
    #                     one_anno = anno[len(anno)//repeat_num * rep:len(anno)//repeat_num * (rep + 1)]
                        one_anno = anno
                        one_X = X
                        AMI.append(adjusted_mutual_info_score(one_anno, one_cluster))
                        ARI.append(adjusted_rand_score(one_anno, one_cluster))
    #                     CPS.append(completeness_score(one_anno, one_cluster))
                        FMI.append(fowlkes_mallows_score(one_anno, one_cluster))
                        NMI.append(normalized_mutual_info_score(one_anno, one_cluster))
    #                     PURITY.append(purity_score(one_anno, one_cluster))
                        # calculate classification metrics for fixed cluster methods only
                        if cluster_file.split('.')[-3] not in methods_non_deterministic_cluster_num:
#                             JACCARD.append(jaccard_score(one_anno, recover_labels(one_anno, one_cluster), average='micro'))
                            ACC.append(accuracy_score(one_anno, recover_labels(one_anno, one_cluster)))
                        else:
#                             JACCARD.append(0)
                            ACC.append(0)
                        # calculate batch metrics for batch datasets only
                        if dataset_dir in dataset_dirs_batch:
                            one_batch = sc_batch_labels
                            if len(np.unique(one_cluster)) != 1:
                                ASW.append(average_silhouette_width(one_X, one_cluster, one_batch))
                            else:
                                ASW.append(0)
                        else:
                            ASW.append(0)
                        # generate 2-D plot according to current best metric
                        # lhs: max metric(e.g ARI) value for the current method, rhs: current metric value
                        if max(np.array(eval(plot_metric))[df.loc[(df.loc[:, 'method'] == group_name) & ([True] * len(eval(plot_metric)) + [False] * (len(df.index) - len(eval(plot_metric)))), :].index.tolist()]) == \
                            adjusted_rand_score(one_anno, one_cluster) and \
                            cluster_file.split('.')[-3] not in methods_non_deterministic_cluster_num:
                            if group_name.endswith("inherit"):
                                plot_metric_df['ngt_imX_unscaled_inherit'] = recover_labels(one_anno, one_cluster)
                            elif group_name.endswith("reinit"):
                                plot_metric_df['ngt_imX_unscaled_reinit'] = recover_labels(one_anno, one_cluster)
                            elif group_name.endswith("inherit-L2"):
                                plot_metric_df['ngt_imX_unscaled_inherit_L2'] = recover_labels(one_anno, one_cluster)
                            elif group_name.endswith("reinit-L2"):
                                plot_metric_df['ngt_imX_unscaled_reinit_L2'] = recover_labels(one_anno, one_cluster)
                            plot_metric_df[group_name] = recover_labels(one_anno, one_cluster)
                print(dataset_dir, "cell#:", len(cluster)//repeat_num)
                df["AMI"] = AMI
                df["ARI"] = ARI
    #             df["CPS"] = CPS
                df["FMI"] = FMI
                df["NMI"] = NMI
#                 df["JACCARD"] = JACCARD
    #             df["PURITY"] = PURITY
                df["ASW"] = ASW
                df["ACC"] = ACC
                output_dir = original_output_dir
#                 df.to_csv(os.path.join(data_dir, dataset_dir, output_dir, "hg%s-dim%d-all.csv" %(hg, dim)))
                # plot
                if plot_flag is True:
                    plot_kinds = ['PCA', 't-SNE', 'UMAP', 'PHATE']
                    pca = PCA(n_components=2)
                    X_r = pca.fit_transform(X)
                    X_embedded = TSNE().fit_transform(X)
                    reducer = umap.UMAP()
                    embedding = reducer.fit_transform(X)
                    phate_op = phate.PHATE(verbose=False)
                    data_phate = phate_op.fit_transform(X)
                    
                    pca_X_unscaled = PCA(n_components=2)
                    X_r_X_unscaled = pca.fit_transform(X_unscaled)
                    X_embedded_X_unscaled = TSNE().fit_transform(X_unscaled)
                    reducer_X_unscaled = umap.UMAP()
                    embedding_X_unscaled = reducer_X_unscaled.fit_transform(X_unscaled)
                    phate_op_X_unscaled = phate.PHATE(verbose=False)
                    data_phate_X_unscaled = phate_op_X_unscaled.fit_transform(X_unscaled)

                    pca_X_normalized = PCA(n_components=2)
                    X_r_X_normalized = pca.fit_transform(X_normalized)
                    X_embedded_X_normalized = TSNE().fit_transform(X_normalized)
                    reducer_X_normalized = umap.UMAP()
                    embedding_X_normalized = reducer_X_normalized.fit_transform(X_normalized)
                    phate_op_X_normalized = phate.PHATE(verbose=False)
                    data_phate_X_normalized = phate_op_X_normalized.fit_transform(X_normalized)
                    
                    imX_inherit = pd.read_csv(os.path.join(data_dir, dataset_dir, output_dir, "hg%s-dim%d-inherit-KMeans.1.imx.txt" %(hg, dim)), index_col=0)

                    pca_imX_inherit = PCA(n_components=2)
                    X_r_imX_inherit = pca.fit_transform(imX_inherit)
                    X_embedded_imX_inherit = TSNE().fit_transform(imX_inherit)
                    reducer_imX_inherit = umap.UMAP()
                    embedding_imX_inherit = reducer_imX_inherit.fit_transform(imX_inherit)
                    phate_op_imX_inherit = phate.PHATE(verbose=False)
                    data_phate_imX_inherit = phate_op_imX_inherit.fit_transform(imX_inherit)
                    
                    imX_reinit = pd.read_csv(os.path.join(data_dir, dataset_dir, output_dir, "hg%s-dim%d-reinit-KMeans.1.imx.txt" %(hg, dim)), index_col=0)

                    pca_imX_reinit = PCA(n_components=2)
                    X_r_imX_reinit = pca.fit_transform(imX_reinit)
                    X_embedded_imX_reinit = TSNE().fit_transform(imX_reinit)
                    reducer_imX_reinit = umap.UMAP()
                    embedding_imX_reinit = reducer_imX_reinit.fit_transform(imX_reinit)
                    phate_op_imX_reinit = phate.PHATE(verbose=False)
                    data_phate_imX_reinit = phate_op_imX_reinit.fit_transform(imX_reinit)
                    
                    imX_inherit_L2 = pd.read_csv(os.path.join(data_dir, dataset_dir, output_dir, "hg%s-dim%d-inherit-KMeans-L2.1.imx.txt" %(hg, dim)), index_col=0)

                    pca_imX_inherit_L2 = PCA(n_components=2)
                    X_r_imX_inherit_L2 = pca.fit_transform(imX_inherit_L2)
                    X_embedded_imX_inherit_L2 = TSNE().fit_transform(imX_inherit_L2)
                    reducer_imX_inherit_L2 = umap.UMAP()
                    embedding_imX_inherit_L2 = reducer_imX_inherit_L2.fit_transform(imX_inherit_L2)
                    phate_op_imX_inherit_L2 = phate.PHATE(verbose=False)
                    data_phate_imX_inherit_L2 = phate_op_imX_inherit_L2.fit_transform(imX_inherit_L2)
                    
                    imX_reinit_L2 = pd.read_csv(os.path.join(data_dir, dataset_dir, output_dir, "hg%s-dim%d-reinit-KMeans-L2.1.imx.txt" %(hg, dim)), index_col=0)

                    pca_imX_reinit_L2 = PCA(n_components=2)
                    X_r_imX_reinit_L2 = pca.fit_transform(imX_reinit_L2)
                    X_embedded_imX_reinit_L2 = TSNE().fit_transform(imX_reinit_L2)
                    reducer_imX_reinit_L2 = umap.UMAP()
                    embedding_imX_reinit_L2 = reducer_imX_reinit_L2.fit_transform(imX_reinit_L2)
                    phate_op_imX_reinit_L2 = phate.PHATE(verbose=False)
                    data_phate_imX_reinit_L2 = phate_op_imX_reinit_L2.fit_transform(imX_reinit_L2)
                    
                    fig, axes = plt.subplots(len(plot_kinds), len(plot_metric_df.columns), figsize=(12, 8))
                    scatter_marker_size = 0.1 # None
                    linewidths = None #0 # None

                    #axes = axes.flatten()
                    # j for columns, i for rows
                    for j in range(len(plot_metric_df.columns)):
                        if j in [0, 5, 8, 11, 14]: # raw X embedding
                            axes[0, j].scatter(X_r[:, 0], X_r[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[1, j].scatter(X_embedded[:, 0], X_embedded[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[2, j].scatter(embedding[:, 0], embedding[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[3, j].scatter(data_phate[:, 0], data_phate[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                        elif j in [1]: # X_unscaled 
                            axes[0, j].scatter(X_r_X_unscaled[:, 0], X_r_X_unscaled[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[1, j].scatter(X_embedded_X_unscaled[:, 0], X_embedded_X_unscaled[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[2, j].scatter(embedding_X_unscaled[:, 0], embedding_X_unscaled[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[3, j].scatter(data_phate_X_unscaled[:, 0], data_phate_X_unscaled[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                        elif j in [2]: # X_normalized
                            axes[0, j].scatter(X_r_X_normalized[:, 0], X_r_X_normalized[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[1, j].scatter(X_embedded_X_normalized[:, 0], X_embedded_X_normalized[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[2, j].scatter(embedding_X_normalized[:, 0], embedding_X_normalized[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[3, j].scatter(data_phate_X_normalized[:, 0], data_phate_X_normalized[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                        elif j in [3, 4]: # gt_imX_inherit ngt_imX_inherit
                            axes[0, j].scatter(X_r_imX_inherit[:, 0], X_r_imX_inherit[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[1, j].scatter(X_embedded_imX_inherit[:, 0], X_embedded_imX_inherit[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[2, j].scatter(embedding_imX_inherit[:, 0], embedding_imX_inherit[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[3, j].scatter(data_phate_imX_inherit[:, 0], data_phate_imX_inherit[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                        elif j in [6, 7]: # gt_imX_reinit ngt_imX_reinit
                            axes[0, j].scatter(X_r_imX_reinit[:, 0], X_r_imX_reinit[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[1, j].scatter(X_embedded_imX_reinit[:, 0], X_embedded_imX_reinit[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[2, j].scatter(embedding_imX_reinit[:, 0], embedding_imX_reinit[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[3, j].scatter(data_phate_imX_reinit[:, 0], data_phate_imX_reinit[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                        elif j in [9, 10]: # gt_imX_inherit_L2 ngt_imX_inherit_L2
                            axes[0, j].scatter(X_r_imX_inherit_L2[:, 0], X_r_imX_inherit_L2[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[1, j].scatter(X_embedded_imX_inherit[:, 0], X_embedded_imX_inherit_L2[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[2, j].scatter(embedding_imX_inherit_L2[:, 0], embedding_imX_inherit_L2[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[3, j].scatter(data_phate_imX_inherit_L2[:, 0], data_phate_imX_inherit_L2[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                        elif j in [12, 13]: # gt_imX_reinit_L2 ngt_imX_reinit_L2
                            axes[0, j].scatter(X_r_imX_reinit_L2[:, 0], X_r_imX_reinit_L2[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[1, j].scatter(X_embedded_imX_reinit_L2[:, 0], X_embedded_imX_reinit_L2[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[2, j].scatter(embedding_imX_reinit_L2[:, 0], embedding_imX_reinit_L2[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)
                            axes[3, j].scatter(data_phate_imX_reinit_L2[:, 0], data_phate_imX_reinit_L2[:, 1], c=plot_metric_df.iloc[:, j], s=scatter_marker_size, linewidths=linewidths)

                        # for the top-most row
                        axes[0, j].set_xlabel(plot_metric_df.columns[j])
                        axes[0, j].xaxis.set_label_position('top')
                        # for the left-most column
                        if j == 0:
                            for i in range(len(plot_kinds)):
                                axes[i, j].set_ylabel(plot_kinds[i])
                        # for all subplots
                        for i in range(len(plot_kinds)):
                            axes[i, j].set_xticks([])
                            axes[i, j].set_yticks([])
#                     fig.suptitle('%s best %s %s' % (dataset_dir, plot_metric, output_dir))
                    fig.suptitle('%s %s' % (dataset_dir, plot_metric))
                    #fig.tight_layout()
#                     plt.subplot_tool()
    #                 fig.supxlabel("clustering method") # require matplotlib 3.4
    #                 fig.supylabel("plotting method") # require matplotlib 3.4
    #                 raise

print("Finish!")

hg0.1-dim400-inherit-KMeans.cluster.txt
hg0.1-dim400-reinit-KMeans.cluster.txt
hg0.1-dim400-inherit-KMeans-L2.cluster.txt
hg0.1-dim400-reinit-KMeans-L2.cluster.txt
hg0.1.raceid3.cluster.txt
hg0.1.sc3.cluster.txt
hg0.1.monocle2.cluster.txt
hg0.1.seurat.cluster.txt
hg0.1.simlr.cluster.txt
hg0.1.drjcc.cluster.txt
hg0.1.scvi.cluster.txt
splatter_batch_2 cell#: 4000


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

hg0.1-dim400-inherit-KMeans.cluster.txt
hg0.1-dim400-reinit-KMeans.cluster.txt
hg0.1-dim400-inherit-KMeans-L2.cluster.txt
hg0.1-dim400-reinit-KMeans-L2.cluster.txt
hg0.1.raceid3.cluster.txt
hg0.1.sc3.cluster.txt
hg0.1.monocle2.cluster.txt
hg0.1.seurat.cluster.txt
hg0.1.simlr.cluster.txt
hg0.1.drjcc.cluster.txt
hg0.1.scvi.cluster.txt
splatter_batch_3 cell#: 6000


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

hg0.1-dim400-inherit-KMeans.cluster.txt
hg0.1-dim400-reinit-KMeans.cluster.txt
hg0.1-dim400-inherit-KMeans-L2.cluster.txt
hg0.1-dim400-reinit-KMeans-L2.cluster.txt
hg0.1.raceid3.cluster.txt
hg0.1.sc3.cluster.txt
hg0.1.monocle2.cluster.txt
hg0.1.seurat.cluster.txt
hg0.1.simlr.cluster.txt
hg0.1.drjcc.cluster.txt
hg0.1.scvi.cluster.txt
splatter_batch_4 cell#: 8000


...

hg0.2-dim400-inherit-KMeans.cluster.txt
hg0.2-dim400-reinit-KMeans.cluster.txt
hg0.2-dim400-inherit-KMeans-L2.cluster.txt
hg0.2-dim400-reinit-KMeans-L2.cluster.txt
hg0.2.raceid3.cluster.txt
hg0.2.sc3.cluster.txt
hg0.2.monocle2.cluster.txt
hg0.2.seurat.cluster.txt
hg0.2.simlr.cluster.txt
hg0.2.drjcc.cluster.txt
hg0.2.scvi.cluster.txt
splatter_batch_3 cell#: 6000


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

hg0.2-dim400-inherit-KMeans.cluster.txt
hg0.2-dim400-reinit-KMeans.cluster.txt
hg0.2-dim400-inherit-KMeans-L2.cluster.txt
hg0.2-dim400-reinit-KMeans-L2.cluster.txt
hg0.2.raceid3.cluster.txt
hg0.2.sc3.cluster.txt
hg0.2.monocle2.cluster.txt
hg0.2.seurat.cluster.txt
hg0.2.simlr.cluster.txt
hg0.2.drjcc.cluster.txt
hg0.2.scvi.cluster.txt
splatter_batch_4 cell#: 8000


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Finish!


In [10]:
plot_metric_df.columns

Index(['gt', 'gt_unscaled', 'gt_normalized', 'gt_imX_unscaled_inherit',
       'ngt_imX_unscaled_inherit', '0.2-400-inherit', 'gt_imX_unscaled_reinit',
       'ngt_imX_unscaled_reinit', '0.2-400-reinit',
       'gt_imX_unscaled_inherit_L2', 'ngt_imX_unscaled_inherit_L2',
       '0.2-400-inherit-L2', 'gt_imX_unscaled_reinit_L2',
       'ngt_imX_unscaled_reinit_L2', '0.2-400-reinit-L2', 'RaceID3', 'SC3',
       'Monocle2', 'SIMLR', 'DRjCC'],
      dtype='object')

In [56]:
dataset_dir

'mouse_embryos'

In [50]:
len(cluster)

504

In [61]:
fig.get_size_inches()

array([6., 4.])

In [62]:
fig.dpi

72.0

In [ ]:
fig.get_size_inches() * fig.dpi